In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, LabelEncoder, OrdinalEncoder
from lazypredict.Supervised import LazyRegressor
from sklearn.linear_model import HuberRegressor, RANSACRegressor, BayesianRidge, OrthogonalMatchingPursuit, LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

In [2]:
df = pd.read_csv(r"C:\Users\DHONI HANIF\OneDrive\Documents\AI_Collection_and_Loss_Reverse_Forecast\data_preparation\EDA\Univariate_bivariate_multivariate\Univariate_Analysis\new_kolektor.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                1000 non-null   int64  
 1   time_to_collect           1000 non-null   int64  
 2   avg_bill_methods          1000 non-null   object 
 3   debtor_volume_handled     1000 non-null   int64  
 4   bill_amount_collected     1000 non-null   int64  
 5   total_actual              1000 non-null   int64  
 6   total_cost                1000 non-null   int64  
 7   success_rate              1000 non-null   float64
 8   collector_age             1000 non-null   int64  
 9   collector_marital_status  1000 non-null   object 
 10  collector_gender          1000 non-null   object 
dtypes: float64(1), int64(7), object(3)
memory usage: 86.1+ KB


In [3]:
for i in df.columns:
    if df[i].dtype == "object":
        df[i] = df[i].astype("category")

df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [4]:
df.head()

,time_to_collect,avg_bill_methods,debtor_volume_handled,bill_amount_collected,total_actual,total_cost,success_rate,collector_age,collector_marital_status,collector_gender
0,0,sms or WA,7,14400000,35700000,277706,40.00,45,Menikah,laki-laki
1,11,panggilan,35,79000000,160500000,22382611,49.00,32,Cerai mati,perempuan
2,4,sms or WA,6,7100000,26700000,205389,27.00,62,Cerai hidup,laki-laki
3,29,datang ke tempat,63,782000000,1395000000,482942548,56.00,55,Cerai mati,perempuan
4,22,datang ke tempat,35,346000000,485500000,151035603,71.00,53,Menikah,perempuan


In [5]:
df["avg_bill_methods"].value_counts()

avg_bill_methods
datang ke tempat    481
sms or WA           198
panggilan           162
surat panggilan     159
Name: count, dtype: int64

In [6]:
bill_methods = LabelEncoder().fit(df["avg_bill_methods"].to_numpy().reshape(-1, 1))
gender = LabelEncoder().fit(df["collector_gender"].to_numpy().reshape(-1, 1))
mart = LabelEncoder().fit(df["collector_marital_status"].to_numpy().reshape(-1, 1))
df2 = pd.DataFrame()

df2["avg_bill_methods"] = bill_methods.transform(df["avg_bill_methods"])
df2["collector_gender"] = gender.transform(df["collector_gender"])
df2["collector_marital_status"] = mart.transform(df["collector_marital_status"])
           
scaler = RobustScaler().fit(df[["bill_amount_collected", "total_actual"]])
df2[["bill_amount_collected", "total_actual"]] = scaler.transform(df[["bill_amount_collected", "total_actual"]])
df2[["time_to_collect", "debtor_volume_handled", "collector_age", "success_rate"]] = df[["time_to_collect", "debtor_volume_handled", "collector_age", "success_rate"]]
y = df["total_cost"]

df2.head(5)

,avg_bill_methods,collector_gender,collector_marital_status,bill_amount_collected,total_actual,time_to_collect,debtor_volume_handled,collector_age,success_rate
0,2,0,3,-0.46,-0.52,0,7,45,40.00
1,1,1,2,-0.28,-0.34,11,35,32,49.00
2,2,0,1,-0.48,-0.53,4,6,62,27.00
3,0,1,2,1.64,1.39,29,63,55,56.00
4,0,1,3,0.45,0.11,22,35,53,71.00


In [7]:
X = df2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
reg = LazyRegressor(verbose=0, ignore_warnings=True)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

  5%|▍         | 2/42 [00:00<00:05,  7.73it/s]

100%|██████████| 42/42 [00:06<00:00,  6.16it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000070 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 760
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 9
[LightGBM] [Info] Start training from score 134930141.780000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
AdaBoostRegressor,0.67,0.68,98210214.57,0.08
PoissonRegressor,0.66,0.67,99537506.92,0.02
BaggingRegressor,0.66,0.67,100144544.91,0.18
RandomForestRegressor,0.65,0.67,100322792.78,1.21
ExtraTreesRegressor,0.63,0.64,104091550.60,0.89
GradientBoostingRegressor,0.62,0.63,105798841.29,0.41
KNeighborsRegressor,0.60,0.62,107211706.57,0.03
LarsCV,0.60,0.61,108343857.33,0.05
LassoLarsCV,0.60,0.61,108400075.78,0.07


In [8]:
y.describe()

count        1000.00
mean    139633105.12
std     166600490.67
min         25263.00
25%       1928856.50
50%      62215627.00
75%     247064723.00
max     678041154.00
Name: total_cost, dtype: float64

In [9]:
def evaluate_performa(X_test, y_test, model1, model2, model3, model4, model5):
    rate = [0, 0, 0, 0, 0]
    #difference = [0, 0, 0, 0, 0]

    X_test = X_test.reset_index(drop=True)
    for i, _ in X_test.iterrows():
        X = np.array(X_test.iloc[[i]])
        y = np.array(y_test.iloc[[i]])
        difference = [0, 0, 0, 0, 0]
        difference[0] = abs(model1.predict(X) - y[0])
        difference[1] = abs(model2.predict(X) - y[0])
        difference[2] = abs(model3.predict(X) - y[0])
        difference[3] = abs(model4.predict(X) - y[0])
        difference[4] = abs(model5.predict(X) - y[0])
        print(difference)

        a = int(np.argmin(difference))
        print(a)
        for i in range(len(difference)):
            if difference[a] == difference[i]:
                rate[i] += 1
    
    a = f"Best model : {rate.index(max(rate))+1}"
    return a, rate

In [10]:
orto = OrthogonalMatchingPursuit()
huber = HuberRegressor()
lr = LinearRegression()
bayes = BayesianRidge()
ransac = RANSACRegressor()

orto.fit(X_train, y_train)
huber.fit(X_train, y_train)
lr.fit(X_train, y_train)
bayes.fit(X_train, y_train)
ransac.fit(X_train, y_train)

a = evaluate_performa(X_test, y_test, orto, huber, lr, bayes, ransac)
a

[array([1.43959229e+08]), array([1.75387913e+08]), array([1.25712133e+08]), array([2.07221754e+08]), array([94687653.48694292])]
4
[array([80987225.34963739]), array([61339561.5861614]), array([45290565.26138592]), array([1.30297162e+08]), array([85278041.6398665])]
2
[array([77954086.6468885]), array([27996774.05314363]), array([54144816.45605171]), array([1.34642313e+08]), array([7463504.82866405])]
4
[array([57648287.26610155]), array([46699964.42261947]), array([49996297.5508702]), array([1.34763313e+08]), array([6565171.77516644])]
4
[array([60028997.78019404]), array([59840635.38603172]), array([79250144.18544044]), array([90962202.83107135]), array([57260641.59654114])]
4
[array([97671540.14033058]), array([4944572.08385393]), array([11642782.68993321]), array([53281802.16848072]), array([74388996.8085647])]
1
[array([1.15542805e+08]), array([70876522.10170588]), array([21346157.87011121]), array([1.33019477e+08]), array([32063231.04638632])]
2
[array([1.58331363e+08]), array([6

('Best model : 5', [27, 34, 54, 16, 69])